# 🦜🔗 LangChain核心源代码解读：关于LLM和Agent（下）

# 课程开始

## 下半场的闲聊

### 🌹 智能体相关类整理

**（1）智能体类型**
- AgentType
    - ZERO_SHOT_REACT_DESCRIPTION（`ReAct`的一般实现）
    - REACT_DOCSTORE（ReAct，支持RAG）
    - SELF_ASK_WITH_SEARCH（使用`search 工具`不断反思获得答案）
    - CONVERSATIONAL_REACT_DESCRIPTION（`ReAct`，支持对话）
    - CHAT_ZERO_SHOT_REACT_DESCRIPTION（同上）
    - CHAT_CONVERSATIONAL_REACT_DESCRIPTION（同上）
    - STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION（`ReAct`，为对话模型优化，支持多输入）
    - OPENAI_FUNCTIONS（支持`OpenAI Function Calling`）
    - OPENAI_MULTI_FUNCTIONS（支持`OpenAI Function Calling`，支持多函数调度）

----
**（2）智能体执行器子组件：**
- `AgentAction`
    - `AgentActionMessageLog`
- `AgentStep`
- `AgentFinish`

----
**（3）单动智能体：**
- `BaseSingleActionAgent`
    - RunnableAgent
    - LLMSingleActionAgent（__deprecated__：`create_***_agent`）
    - XMLAgent（__deprecated__：create_xml_agent）
    - Agent
        - ChatAgent（__deprecated__：`create_react_agent`）
        - ConversationalAgent（__deprecated__：`create_react_agent`）
        - ConversationalChatAgent（__deprecated__：`create_json_chat_agent`）
        - StructuredChatAgent（__deprecated__：`create_structured_chat_agent`）
        - ZeroShotAgent（__deprecated__：`create_react_agent`）
        - ReActDocstoreAgent（__deprecated__）
            - ReActTextWorldAgent（__deprecated__）
        - SelfAskWithSearchAgent（__deprecated__：`create_self_ask_with_search`）
    - OpenAIFunctionsAgent（__deprecated__：`create_openai_functions_agent`）

----
**（4）多动智能体：**
- BaseMultiActionAgent
    - RunnableMultiActionAgent
    - OpenAIMultiFunctionsAgent（__deprecated__：`create_openai_tools_agent`）

----
**（5）Assistant：**
- Runnable
    - RunnableSerializable
        - Chain
            - `AgentExecutor`
                - MRKLChain（__deprecated__）
                - ReActChain（__deprecated__）
                - SelfAskWithSearchChain（__deprecated__）
        - `OpenAIAssistantRunnable`

----
**（6）提示语模板：**
- Runnable
    - RunnableSerializable
        - BasePromptTemplate [Dict, PromptValue]
            - BaseChatPromptTemplate
                - ChatPromptTemplate
                    - `AgentScratchPadChatPromptTemplate`

----
**（7）输出解析：**
- Runnable
    - RunnableSerializable
        - BaseOutputParser
            - `AgentOutputParser`
            - `MultiActionAgentOutputParser`

### 🌹 本节涉及的 LangChain 智能体

- 🌹 [LangChain 中的 Agent](https://python.langchain.com/docs/modules/agents/agent_types/)

|langchain智能体|支持多工具|支持多参数|推理过程优化|
|:---|:---:|:---:|:---:|
|create_openai_tools_agent|支持|支持|依赖大模型能力|
|create_react_agent|不支持|不支持|可以支持|
|自定义CoT|可以支持|可以支持|可以支持|

### 🌹 本节涉及的 LangChain 智能体相关源码

- 🌹 [查看 langchain/agents/openai_tools/base.py 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)
- 🌹 [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)
- 🌹 [查看 create_react_agent 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L16)
- 🌹 [查看 ReAct 输出解析的源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py#L22-L95)
- 🌹 [查看 AgentExcutor 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/agent.py#L1413-L1458)
- 🌹 [Chain](https://python.langchain.com/docs/modules/chains)

# （二）解读源码，拆解和自定义智能体

<div class="alert alert-info">
    <b>干货从这里开始！</b><br>
    接下来的例子中，会穿插 langchian 源码解读。
</div>

In [816]:
# 加载 .env 到环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# 《手撕 AutoGPT》中的有趣小工具
from colorama import init, Fore, Back, Style
import sys

THOUGHT_COLOR = Fore.GREEN
OBSERVATION_COLOR = Fore.YELLOW
ROUND_COLOR = Fore.RED
CODE_COLOR = Fore.BLUE

def color_print(text, color=None, end="\n"):
    if color is not None:
        content = color + text + Style.RESET_ALL + end
    else:
        content = text + end
    sys.stdout.write(content)
    sys.stdout.flush()

## 4、解读 OpenAI 工具回调风格智能体

### ✍️ create_openai_executor：拆解 OpenAI 工具回调智能体的定义过程

![还是那个故事，但这次我们让老大爷聪明一点...如果可以的话。](./madongmei.gif)

In [817]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain import hub
from langchain.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool, convert_to_openai_function
import re

@tool
def ask_neighber(query: str) -> str:
    """想问你找的人住哪里就问我吧，我是楼下老大爷"""
    if(re.search("马冬梅", query)):
        return "马冬梅住在楼上322。"
    else:
        return "我不清楚"

def create_openai_executor(llm, tools):
    """
    使用openai智能体定义一个应用
    """
    # 定义 prompt
    prompt = hub.pull("hwchase17/openai-tools-agent")
    # 定义 Agent
    agent = create_openai_tools_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=False)

    return executor

neighber = create_openai_executor(ChatZhipuAI(), [ask_neighber])

### 🌹 prompt： 观察 OpenAI 智能体的提示语

#### （1）从 Langsmith 的 hub 下载 hwchase17/openai-tools-agent

[查看 hub.pull("hwchase17/openai-tools-agent")](https://smith.langchain.com/hub/hwchase17/openai-tools-agent)

#### （2）等价的自定义 Prompt 模板

In [818]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

In [819]:
prompt.invoke({
    "input": "请问马冬梅的家在哪里？",
    "chat_history": [],
    "agent_scratchpad": []
})

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='请问马冬梅的家在哪里？')])

### 🌹 agent：阅读 create_openai_tools_agent 源码

[查看 langchain/agents/openai_tools/base.py 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/openai_tools/base.py#L15-L97)

```python
def create_openai_tools_agent(
    llm: BaseLanguageModel, tools: Sequence[BaseTool], prompt: ChatPromptTemplate
) -> Runnable:
    """Create an agent that uses OpenAI tools."""

    missing_vars = {"agent_scratchpad"}.difference(prompt.input_variables)
    if missing_vars:
        raise ValueError(f"Prompt missing required variables: {missing_vars}")

    llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in tools])

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_tool_messages(
                x["intermediate_steps"]
            )
        )
        | prompt
        | llm_with_tools
        | OpenAIToolsAgentOutputParser()
    )
    return agent
```

### 🌹 executor： 阅读 AgentExcutor 源码

[查看 AgentExcutor 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/agent.py#L1413-L1458)

- 重点看 `_call` 方法实现
- 作为 Chain 子类，AgentExcutor 覆写了 stream 方法。
- 代码中的运行逻辑非常复杂，因此我们可以通过下面的方式研究其实际运行过程


### 🌹 run：观察 OpenAI 智能体的运行过程

#### （1）简单执行：invoke

In [820]:
# invoke
neighber.invoke({"input":"马冬梅住哪里"})



> Entering new AgentExecutor chain...

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


马冬梅住在楼上322。根据楼下老大爷的信息，马冬梅住在楼上322。

> Finished chain.


{'input': '马冬梅住哪里', 'output': '根据楼下老大爷的信息，马冬梅住在楼上322。'}

#### （2）流输出：stream（仅智能体中的流）

In [114]:
# stream
for s in neighber.stream({"input":"马冬梅住哪里"}):
    print(s)



> Entering new AgentExecutor chain...
{'actions': [OpenAIToolAgentAction(tool='ask_neighber', tool_input={'query': '马冬梅住哪里'}, log="\nInvoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516974814413003915', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})], tool_call_id='call_8516974814413003915')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516974814413003915', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})]}

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322{'steps': [AgentStep(action=OpenAIToolAgentAction(tool='ask_neighber', tool_input={'query': '马冬梅住哪里'}, log="\nInvoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': '

#### （3）还是要使用事件流：astream_events

**看看方法 astream_events 的能力：**

In [14]:
# astream_events
async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print(e['name'], e['tags'], e['event'])

AgentExecutor

> Entering new AgentExecutor chain...
 [] on_chain_start
RunnableSequence [] on_chain_start
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_start
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_stream
RunnableParallel<agent_scratchpad> [] on_chain_start
RunnableLambda ['map:key:agent_scratchpad'] on_chain_start
RunnableLambda ['map:key:agent_scratchpad'] on_chain_stream
RunnableParallel<agent_scratchpad> [] on_chain_stream
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_stream
RunnableLambda ['map:key:agent_scratchpad'] on_chain_end
RunnableParallel<agent_scratchpad> [] on_chain_end
RunnableAssign<agent_scratchpad> ['seq:step:1'] on_chain_end
ChatPromptTemplate ['seq:step:2'] on_prompt_start
ChatPromptTemplate ['seq:step:2'] on_prompt_end
ChatZhipuAI ['seq:step:3'] on_chat_model_start
ChatZhipuAI ['seq:step:3'] on_chat_model_stream
ChatZhipuAI ['seq:step:3'] on_chat_model_stream
ChatZhipuAI ['seq:step:3'] on_chat_model_end
OpenAIToolsAgentOutpu

#### （4）观察与大模型的交互过程

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [34]:
from langchain_zhipu import ChatZhipuAI
neighber = create_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])
        # print("\n", e)



> Entering new AgentExecutor chain...

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:
{'messages': [[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='马冬梅住哪里')]]}

 ---------- ChatZhipuAI -- on_chat_model_end ----------
OUTPUT:
{'generations': [[{'text': '', 'generation_info': {'finish_reason': 'tool_calls'}, 'type': 'ChatGenerationChunk', 'message': AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_8516985431572319825', 'function': {'arguments': '{"query":"马冬梅住哪里"}', 'name': 'ask_neighber'}, 'type': 'function'}]})}]], 'llm_output': None, 'run': None}

Invoking: `ask_neighber` with `{'query': '马冬梅住哪里'}`


楼上322
 ---------- ask_neighber -- on_tool_end ----------
INPUT:
{'query': '马冬梅住哪里'}

 ---------- ask_neighber -- on_tool_end ----------
OUTPUT:
楼上322

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:
{'messages': [[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='马冬

#### （5）总结智能体的定义过程

- STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求
- STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息
- STEP-3 调用本地工具（langchain）: 调用工具
- STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果
- STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

### ✍️ 一步一步执行：就像我们常对大模型说的那样

#### （1）STEP-1 请求智能体（langchain -> ZhipuAI）: 发送带有Tools的请求

In [97]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

# openai agent
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant"),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

input = "马冬梅住在哪里？"
history = []

llm = ChatZhipuAI(tools=[convert_to_openai_tool(ask_neighber)])

agent = prompt | llm

<div class="alert alert-warning">
    <b>⚠️ 思考：</b><br>
    <b>convert_to_openai_function</b> 和 <b>convert_to_openai_tool</b> 的区别是什么？
</div>

**🌞 参考：**
- [查看 convert_to_openai_tool 的实现源码](https://github.com/langchain-ai/langchain/blob/c93d4ea91cfcf55dfe871931d42aa22562f8dae2/libs/core/langchain_core/utils/function_calling.py#L323-L341)


In [38]:
convert_to_openai_function(ask_neighber)

{'name': 'ask_neighber',
 'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [39]:
convert_to_openai_tool(ask_neighber)

{'type': 'function',
 'function': {'name': 'ask_neighber',
  'description': 'ask_neighber(query: str) -> str - 我在玩找人的游戏，我知道你找的人住在哪个房间',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string'}},
   'required': ['query']}}}

#### （2）STEP-2 智能体解析（ZhipuAI -> langchain）: 收到Tools-Calling消息

In [104]:
steps_info = []
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8516987149559268121', 'function': {'arguments': '{"query":"马冬梅住在哪里？"}', 'name': 'ask_neighber'}, 'type': 'function'}]})

#### （3）STEP-3 调用本地工具（langchain）: 调用工具

In [105]:
resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"]

'{"query":"马冬梅住在哪里？"}'

In [107]:
import json
tool_args = json.loads(resp_llm.additional_kwargs["tool_calls"][0]["function"]["arguments"])
tool_args

{'query': '马冬梅住在哪里？'}

In [108]:
resp_tool = ask_neighber.invoke(tool_args)
resp_tool

'楼上322'

#### （4）STEP-4 重新请求智能体（langchain） -> ZhipuAI）: 提交调用结果

In [110]:
steps_info.append(input)
steps_info.append(resp_llm)
steps_info.append(resp_tool)
steps_info

['马冬梅住在哪里？',
 '楼上322',
 '马冬梅住在哪里？',
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8516987149559268121', 'function': {'arguments': '{"query":"马冬梅住在哪里？"}', 'name': 'ask_neighber'}, 'type': 'function'}]}),
 '楼上322']

#### （5）STEP-5 智能体最终生成（ZhipuAI -> langchain）: 大模型重新生成结果

In [111]:
resp_llm = agent.invoke({"input": input, "chat_history": history, "agent_scratchpad": steps_info})
resp_llm

AIMessage(content='根据我的查询结果，马冬梅住在楼上322。您还需要我提供其他信息吗？')

### 🌹 从源码中看 openai 作为先驱的影响

- [partners/openai/langchain_openai/chat_models/base.py（OpenAI模型）](https://github.com/langchain-ai/langchain/blob/master/libs/partners/openai/langchain_openai/chat_models/base.py)
- [community/langchain_community/adapters/openai.py（OpenAI风格API转换）](https://github.com/langchain-ai/langchain/blob/master/libs/community/langchain_community/adapters/openai.py)
- [core/langchain_core/utils/function_calling.py（OpenAI风格回调工具函数）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/utils/function_calling.py)
- [core/langchain_core/output_parsers/openai_tools.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_tools.py)
- [core/langchain_core/output_parsers/openai_functions.py（OpenAI风格输出解析）](https://github.com/langchain-ai/langchain/blob/master/libs/core/langchain_core/output_parsers/openai_functions.py)

## 5、解读 ReAct 风格智能体

### 🌹 构造 ReAct 智能体的提示语

[查看 hub.pull("hwchase17/react")](https://smith.langchain.com/hub/hwchase17/react)

In [149]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder

In [150]:
prompt = hub.pull("hwchase17/react")
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### ✍️ create_react_executor：拆解 react 智能体的核心结构

#### （1）定义 ReAct 智能体

<div class="alert alert-info">
    <b>⚠️ 注意：</b><br>
    ReAct 智能体提示语的核心在于：<b>请使用如下的输出格式...</b>
    <br>
    这相当于提示语模板中的：step by step ...
</div>

In [573]:
# from langchain_openai import ChatOpenAI
from langchain_zhipu import ChatZhipuAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain import hub
from langchain.tools import tool
import re

@tool
def ask_neighber(query: str) -> str:
    """我是马冬梅的邻居老大爷，关于她的事情你可以问我"""
    if(re.search("马冬梅", query)):
        return "楼上322"
    else:
        return "我不清楚"

prompt_react = """
请尽你最大努力回答用户的问题。

你必须注意以下原则：
1. 在需要使用本地文件时，请务必使用相关工具查询，而不要编造文件名。
2. 请使用中文。

你可以访问如下工具：
{tools}

请使用如下的输出格式：

Question: 你必须回答的问题
Thought: 你为了完成任务必须采取的思考过程
Action: 又称为工具，必须是这些工具其中之一： {tool_names}
Action Input: 调用工具所使用的参数
Observation: 工具执行的结果
... (这些过程 Thought/Action/Action Input/Observation 可以重复执行N次)
Thought: 我现在知道答案了
Final Answer: 问题的最终答案

开始！

Question: {input}
Thought:{agent_scratchpad}
"""

def create_react_executor(llm, tools):
    # 定义 prompt
    prompt = PromptTemplate.from_template(prompt_react)
    # 定义 Agent
    agent = create_react_agent(llm, tools, prompt)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=False)
    
    return executor

neighber = create_react_executor(ChatZhipuAI(), [ask_neighber])

#### （2）简单执行：invoke

In [570]:
# invoke
neighber.invoke({"input":"马冬梅住哪个房间"})



> Entering new AgentExecutor chain...
我需要获取马冬梅住的房间信息，这个问题明显需要询问她的邻居老大爷。

Action: ask_neighber
Action Input: '马冬梅住哪个房间'
Observation楼上322Thought: 我已经从邻居老大爷那里得到了马冬梅的房间号，现在我知道答案了。

Final Answer: 马冬梅住在楼上322房间。

> Finished chain.


{'input': '马冬梅住哪个房间', 'output': '马冬梅住在楼上322房间。'}

#### （3）大模型的流输出：stream VS astream_events

In [574]:
for chunk in neighber.stream({"input": "马冬梅住哪个房间"}):
    print(chunk)



> Entering new AgentExecutor chain...
{'actions': [AgentAction(tool='ask_neighber', tool_input="'马冬梅住哪个房间'\nObservation", log="我需要获取马冬梅住哪个房间的信息，这个问题明显需要询问她的邻居老大爷。\n\nAction: ask_neighber\nAction Input: '马冬梅住哪个房间'\nObservation")], 'messages': [AIMessage(content="我需要获取马冬梅住哪个房间的信息，这个问题明显需要询问她的邻居老大爷。\n\nAction: ask_neighber\nAction Input: '马冬梅住哪个房间'\nObservation")]}
我需要获取马冬梅住哪个房间的信息，这个问题明显需要询问她的邻居老大爷。

Action: ask_neighber
Action Input: '马冬梅住哪个房间'
Observation楼上322{'steps': [AgentStep(action=AgentAction(tool='ask_neighber', tool_input="'马冬梅住哪个房间'\nObservation", log="我需要获取马冬梅住哪个房间的信息，这个问题明显需要询问她的邻居老大爷。\n\nAction: ask_neighber\nAction Input: '马冬梅住哪个房间'\nObservation"), observation='楼上322')], 'messages': [HumanMessage(content='楼上322')]}
Thought: 我已经从邻居老大爷那里得到了马冬梅的房间号，现在我知道答案了。

Final Answer: 马冬梅住在楼上322房间。

> Finished chain.
{'output': '马冬梅住在楼上322房间。', 'messages': [AIMessage(content='Thought: 我已经从邻居老大爷那里得到了马冬梅的房间号，现在我知道答案了。\n\nFinal Answer: 马冬梅住在楼上322房间。')]}


In [575]:
async for event in neighber.astream_events({"input": "马冬梅住哪里"}, version="v1"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="_")
    else:
        print(kind)



> Entering new AgentExecutor chain...
on_chain_start
on_chain_start
on_chain_start
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_stream
on_chain_stream
on_chain_end
on_chain_end
on_chain_end
on_prompt_start
on_prompt_end
on_chat_model_start
我_需要_获取_马_冬_梅_的_住_址_信息_，_这个问题_明显_需要_使用_邻居_老大_爷_提供的_工具_ask__ne_igh_ber_。

Action_:_ ask__ne_igh_ber_
Action_ Input_:_ query_='_马_冬_梅_住_在哪里_'
Observ_ation__on_chat_model_end
on_parser_start
on_parser_end
on_chain_stream
on_chain_end
on_chain_stream
我需要获取马冬梅的住址信息，这个问题明显需要使用邻居老大爷提供的工具ask_neighber。

Action: ask_neighber
Action Input: query='马冬梅住在哪里'
Observationon_tool_start
楼上322on_tool_end
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_start
on_chain_start
on_chain_stream
on_chain_stream
on_chain_stream
on_chain_end
on_chain_end
on_chain_end
on_prompt_start
on_prompt_end
on_chat_model_start
Thought_:_ 通过_邻居_老大_爷_，_我已经_获得了_马_冬_梅_的_住_址_信息_。

Final_ Answer_:_ 马_冬_梅_住在_楼上_3_22_。__on_chat_model_end
on_parser

### 🌹 观察 ReAct 智能体的运行过程

只读取 **on_tool_end** 和 **on_chat_model_end** 两个事件：

In [577]:
from langchain_zhipu import ChatZhipuAI
neighber = create_react_neighber(ChatZhipuAI())

async for e in neighber.astream_events({"input":"马冬梅住哪里"}, version="v1"):
    print_line = lambda : print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            input = e['data']['input']
            print_line()
            print("INPUT:")
            if(input is not None and "messages" in input):
                print(input["messages"][0][0].content)
            else:
                print(input)
        if("output" in e['data']):
            output = e['data']['output'] 
            print_line()
            print("OUTPUT:")
            if(output is not None and "generations" in output):
                print(output["generations"][0][0]["text"])
            else:
                print(output)
        # print("\n", e)



> Entering new AgentExecutor chain...

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:

请尽你最大努力回答用户的问题。

你必须注意以下原则：
1. 在需要使用本地文件时，请务必使用相关工具查询，而不要编造文件名。
2. 请使用中文。

你可以访问如下工具：
ask_neighber: ask_neighber(query: str) -> str - 我是马冬梅的邻居老大爷，关于她的事情你可以问我

请使用如下的输出格式：

Question: 你必须回答的问题
Thought: 你为了完成任务必须采取的思考过程
Action: 又称为工具，必须是这些工具其中之一： ask_neighber
Action Input: 调用工具所使用的参数
Observation: 工具执行的结果
... (这些过程 Thought/Action/Action Input/Observation 可以重复执行N次)
Thought: 我现在知道答案了
Final Answer: 问题的最终答案

开始！

Question: 马冬梅住哪里
Thought:


 ---------- ChatZhipuAI -- on_chat_model_end ----------
OUTPUT:
我需要通过询问马冬梅的邻居来获取她的住址信息。

Action: ask_neighber
Action Input: '马冬梅住在哪里？'
Observation
我需要通过询问马冬梅的邻居来获取她的住址信息。

Action: ask_neighber
Action Input: '马冬梅住在哪里？'
Observation楼上322
 ---------- ask_neighber -- on_tool_end ----------
INPUT:
'马冬梅住在哪里？'
Observation

 ---------- ask_neighber -- on_tool_end ----------
OUTPUT:
楼上322

 ---------- ChatZhipuAI -- on_chat_model_end ----------
INPUT:

请尽你最大努力回答用户

### 🌹 阅读源码，解读 ReAct 智能体

- `ReAct` 比 `OpenAI智能体` 多了必要的解析过程。

- [查看 create_react_agent 源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L16)
- [查看 ReAct 输出解析的源码](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py#L22-L95)

<div class="alert alert-warning">
    <b>思考：上面的提示语中为什么不修改这几个关键词？</b><br>
    <ul>
        <li>Final Answer</li>
        <li>Action</li>
        <li>Action Input</li>
        <li>Observation</li>
    </ul>
</div>

## 6、在LCEL框架下重定义《手撕AutoGPT》中的智能体

### ✍️ 自定义基于思维链的提示语

<div class="alert alert-info">
    <b>⚠️ 注意：</b><br>
    CoT 智能体提示语的核心在于：<b>请使用如下的输出格式...</b>
    <br>
    这相当于提示语模板中的：step by step ...
</div>

In [578]:
PROMPT_COT = """
你是强大的AI助手，可以使用工具与指令自动化解决问题。

你必须遵循以下约束来完成任务:
1. 每次你的决策只使用一种工具，你可以使用任意多次。
2. 确保你调用的指令或使用的工具在下述给定的工具列表中。
3. 确保你的回答不会包含违法或有侵犯性的信息。
4. 如果你已经完成所有任务，确保以"FINISH"指令结束。
5. 用中文思考和输出。
6. 如果执行某个指令或工具失败，尝试改变参数或参数格式再次调用。
7. 你生成的回复必须遵循上文中给定的事实信息。不可以编造信息。DO NOT MAKE UP INFORMATION.
8. 如果得到的结果不正确，尝试更换表达方式。
9. 已经得到的信息，不要反复查询。
10. 确保你生成的动作是可以精确执行的。动作做中可以包括具体方法和目标输出。
11. 看到一个概念时尝试获取它的准确定义，并分析从哪些输入可以得到它的具体取值。
12. 生成一个自然语言查询时，请在查询中包含全部的已知信息。
13. 在执行分析或计算动作前，确保该分析或计算中涉及的所有子概念都已经得到了定义。
14. 你不可以打印一个文件的全部内容，这样的操作代价太大，且会造成不可预期的后果，是被严格禁止的。
15. 不要向用户提问。
16. 在需要使用本地文件时，请务必使用相关工具查询，而不要编造文件名。

你的任务是:
{input}

你有非常优秀的逻辑分析能力，可以通过因果关系找到最优的解决方案。

你要参考之前的思考记录:
{agent_scratchpad}

你需要评估你的表现:
1. 尽你最大的努力，用你最好的水平，通过分析和检查，做出最好的决定。
2. 带着全局观，自我反思你计划与动作。
3. 考虑你之前的策略与决策来改善的你的计划。
4. 如果你反复得到相同的结果，修改你的计划和决策，避免死循环。
5. 如果你当前的动作无法获取到需要的信息，尝试展开关键概念的定义，再重新推理。

如果你必须选择工具才能完成任务，可以使用以下工具之一，它们又称为动作或actions:

{tools}

你必须根据以下格式说明，输出你的思考过程:
1. 关键概念: 任务中涉及的组合型概念或实体。已经明确获得取值的关键概念，将其取值完整备注在概念后。
2. 概念拆解: 将任务中的关键概念拆解为一系列待查询的子要素。每个关键概念一行，后接这个概念的子要素，每个子要素一行，行前以' -'开始。
3. 反思:
   - 自我反思，观察以前的执行记录，思考概念拆解是否完整、准确。
   - 一步步思考是否每一个的关键概念或要素的查询都得到了准确的结果。
   - 反思你已经得到哪个要素/概念。你得到的要素/概念取值是否正确。从当前的信息中还不能得到哪些要素/概念。
   - 每个反思一行，行前以' -'开始。
4. 思考: 观察执行记录和你的自我反思，并一步步思考
  （1）分析要素间的依赖关系，例如：
    i. 我是否需要先获得A的值/定义，才能通过A来获得B？
    ii. 如果我先获得A，是否可以通过A筛选B，减少穷举每个B的代价？
    iii. A和B是否存在在同一数据源中，我能否在获取A的同时获取B？
    iv. 是否还有更高效或更聪明的办法来查询一个概念或要素？
    v. 如果上一次尝试查询一个概念或要素时失败了，我是否可以尝试从另一个资源中再次查询？
    vi. 诸如此类，你可以扩展更多的思考 ...
  （2）根据以上分析，排列子要素间的查询优先级
  （3）找出当前需要获得取值的子要素
  注意，不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源！
5. 推理: 根据你的反思与思考，一步步推理被选择的子要素取值的获取方式。如果前一次的计划失败了，请检查输入中是否包含每个概念/要素的明确定义，并尝试细化你的查询描述。
6. 计划: 严格遵守以下规则，计划你的当前动作。
  （1）详细列出当前动作的执行计划。只计划一步的动作。PLAN ONE STEP ONLY!
  （2）一步步分析，包括数据源，对数据源的操作方式，对数据的分析方法。有哪些已知常量可以直接代入此次分析。
  （3）不要尝试计算文件的每一个元素，这种计算代价太高，是严格禁止的。你可以通过分析找到更有效的方法，比如条件筛选。
  （4）上述分析是否依赖某个要素的取值/定义，且该要素的取值/定义尚未获得。若果是，重新规划当前动作，确保所有依赖的要素的取值/定义都已经获得。
  （5）不要对要素的取值/定义做任何假设，确保你的信息来自给定的数据源。不要编造信息。DO NOT MAKE UP ANY INFORMATION!!!
  （6）确保你执行的动作涉及的所有要素都已获得确切的取值/定义。
  （7）如果全部子任务已完成，请用FINISH动作结束任务。
  （8）不要在这里使用JSON格式来描述计划的当前动作，这会与下面的输出重复，造成无法解析。

你必须根据以下格式说明，输出JSON格式，描述所选择执行的动作/工具/指令:
{action_format_instructions}
"""

### ✍️ 自定义 OutputParser

In [579]:
from typing import List, Optional, Dict, Any, Union, Callable
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.agent import AgentOutputParser, AgentAction, AgentFinish
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.output_parsers import PydanticOutputParser

In [580]:
class Action(BaseModel):
    name: str = Field(
        description="The name of tool or action: FINISH or Other tool names."
        )
    args: Optional[Dict[str, Any]] = Field(
        default=None,
        description="Parameters of tool or action are composed of names and values."
        )

# 解析Action
_action_output_parser = PydanticOutputParser(pydantic_object=Action)
_action_parser_format = _action_output_parser.get_format_instructions()

class ReasonOutputParser(AgentOutputParser):
    """解析单个动作的智能体action和输入参数。
    """

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        action: Action = _action_output_parser.invoke(text)
        name: Optional[str] = action.name
        args: Optional[Dict[str, Any]] = action.args if text is not None else "No Args"
        log: str = text if text is not None else ""

        if name == "FINISH":
            return AgentFinish(args, log)
        elif name is not None:
            return AgentAction(name, args, log)

    @property
    def _type(self) -> str:
        return "Chain-of-Thought"


<div class="alert alert-warning">
    <b>💡 思考：自定义的CoT智能体，与ReAct智能体在判断推理结束方面的策略差别是什么？</b>
</div>

### ✍️ FINISH

In [581]:
@tool
def FINISH(output: str) -> str:
    """
    用于表示任务完成的占位符工具。
    Args:
        output - 这是你要输出的答案。
    """
    
    return output

### ✍️ 自定义一个智能体

In [582]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description

def _prompt_creator(prompt: str) -> Callable[[List[str]], str]:
    def creator(tools: List[str]) -> str:
        # 请注意，智谱AI等国内大模型对于pydantic的参数解析并不友好，使用JSON描述参数时会误读
        # 因此，不要使用 render_text_description_and_args 来生成工具描述
        tools_format = render_text_description(tools)

        template = PromptTemplate.from_template(prompt)
        return template.partial(
            tools=tools_format,
            action_format_instructions=_action_parser_format,
        )

    return creator

# 基于 CoT 的智能体
def create_cot_agent(llm: Any, prompt: Optional[str] = None, tools: List[str] = []) -> Any:
    prompt_creator = _prompt_creator(PROMPT_COT)
    if prompt is not None:
        prompt_creator = _prompt_creator(prompt)

    agent = (
        RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"])
        )
        | prompt_creator(tools)
        | llm
        | ReasonOutputParser()
    )

    return agent

### ✍️ create_cot_executor：尝试运行

In [583]:
def create_cot_executor(llm, tools):
    # 定义 Agent
    agent = create_cot_agent(llm, tools=tools)
    # 定义 AgentExecutor
    executor = AgentExecutor(agent=agent, tools=tools, verbose=False, handle_parsing_errors=False)
    
    return executor

neighber = create_cot_executor(ChatZhipuAI(), [ask_neighber, FINISH])

In [585]:
async for chunk in neighber.astream_events({"input": "马冬梅家住哪个房间？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

```_json_
{
 _ "_name_":_ "_ask__ne_igh_ber_",
 _ "_args_":_ {
   _ "_query_":_ "_马_冬_梅_家住_哪个_房间_？_"
 _ }
}
```__```_json_
{
 _ "_name_":_ "_FIN_ISH_",
 _ "_args_":_ {
   _ "_output_":_ "_马_冬_梅_家住_楼上_3_22_房间_"
 _ }
}
```_ 

【_关键_概念_】_
-_ 马_冬_梅_家_房间_号_

【_概念_拆_解_】_
-_ 马_冬_梅_家_房间_号_ -_ 待_查询_

【_反思_】_
-_ 我_已经_通过_询问_邻居_得到了_马_冬_梅_的_房间_号_是_楼上_3_22_。

【_思考_】_
-_ 既然_已经_获得了_马_冬_梅_的_房间_号_，_就没有_必要_再_进行_其他_查询_。

【_推理_】_
-_ _之前_通过_工具_ ask__ne_igh_ber_ 获_得了_所需_信息_，_现在_可以直接_用_ FIN_ISH_ 结_束_任务_。

【_计划_】_
-_ 使用_ FIN_ISH_ 工_具_结束_任务_，_并_输出_已知_信息_：“_马_冬_梅_家住_楼上_3_22_房间_”。__

# （三）再现《手撕AutoGPT》：langchain+智谱+智能体

## 7、定义工具

### ✍️ 列举文件

In [586]:
work_dir = "./data"

In [587]:
# from langchain_openai import ChatOpenAI
from langchain.tools import tool
import os
import re
import fnmatch

@tool
def list_files(args=None) -> str:
    """如果需要查询资料，就首先使用该工具探查本地文件夹的结构和内容，展示它的文件名和文件夹名"""

    # 工作目录
    print("当前目录为：", workd_dir)

    # 定义你想要过滤的模式
    patterns = ['*.pdf', "*.xlsx"]

    if(os.path.isdir(workd_dir)):
        all_files = os.listdir(work_dir)
        # 过滤出所有匹配的文件
        matching_files = [f for f in all_files for p in patterns if fnmatch.fnmatch(f, p)]

        return "\n".join(matching_files)
    else:
        return []

In [588]:
print(list_files.invoke({}))

当前目录为： ./data
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
供应商资格要求.pdf


### ✍️ 查询文档

#### （1）准备

In [654]:
from typing import List
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_zhipu import ChatZhipuAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.messages import BaseMessage

In [655]:
def get_file_extension(filename: str) -> str:
    return filename.split(".")[-1]

def format_docs(docs: List[str]) -> str:
    return "\n\n".join([d.page_content for d in docs])

def convert_message_to_str(message: Union[BaseMessage, str]) -> str:
    if isinstance(message, BaseMessage):
        return message.content
    else:
        return message

In [656]:
class FileLoadFactory:
    @staticmethod
    def get_loader(filename: str):
        filename = filename.strip()
        ext = get_file_extension(filename)
        if ext == "pdf":
            return PyPDFLoader(filename)
        elif ext == "docx" or ext == "doc":
            return Docx2txtLoader(filename)
        else:
            raise NotImplementedError(f"File extension {ext} not supported.")

In [657]:
def load_docs(filename: str) -> List[Document]:
    file_loader = FileLoadFactory.get_loader(filename)
    return file_loader.load_and_split()

#### （2）使用 RAG 查询文档

In [777]:
@tool
def ask_document(
        filename: str,
        query: str,
) -> str:
    """
    查询Word或PDF文档中的文本内容，以便回答问题。
    考虑上下文信息，确保问题对相关概念的定义表述完整。
    """

    path = os.path.join(work_dir, filename)
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"

    chunks = load_docs(path)
    # print(chunks)
    if chunks is None or len(chunks) == 0:
        return "无法读取文档内容"

    db = Chroma.from_documents(chunks, OpenAIEmbeddings())

    DEFAULT_QA_CHAIN_PROMPT = """
        你要严格依据如下资料回答问题，你的回答不能与其冲突，更不要编造。
        请始终使用中文回答。
        
        {context}
        
        问题: {question}
        """
    prompt = ChatPromptTemplate.from_template(DEFAULT_QA_CHAIN_PROMPT)

    qa_chain = (
        {
            "context": (lambda x: convert_message_to_str(x)) | db.as_retriever() | format_docs,
            "question": lambda x: convert_message_to_str(x)
        }
        | prompt
        | ChatZhipuAI()
    )

    # response = qa_chain.invoke(query)
    final_output = ""
    for chunk in qa_chain.stream(query):
        print(chunk.content, end="|")
        final_output += chunk.content
        
    return final_output

<div class="alert alert-warning">
    <b>思考：BaseTool 的定义中没有流方法，所以工具定义中也没有流输出支持</b><br>
    这将导致智能体在调用时，实际上无法获得流式输出（尽管可以在后台调试时打印，但不方便传递）。<br>
    这是为什么呢？？
</div>

In [778]:
ask_document.invoke({"filename": "供应商资格要求.pdf", "query": "供应商达标标准"})

供应商|要|达到|的标准|主要包括|以下|六个|方面|：

一|、|基本|资质|要求|：
  | |1|.| 必|须|具备|合法|有效的|营业执照|。
  | |2|.| 必|须|具备|税务|登记|证|，|且|符合|国家|税收|法律法规|。
  | |3|.| 应|拥有|有效的|组织|机构|代码|证|。

二|、|经营|和|财务|能力|：
  | |1|.| 与|公司|合作的|月份|，|销售|的产品|月|总价|不得|低于|人民币|3|万元|。
  | |2|.| 具|备|健全|和|稳定的|财务|状况|，|以及|良好的|信用|记录|。
  | |3|.| 具|备|足够的|流动|资金|来|支持|合同|执行|。

三|、|产品|和服务|质量|：
  | |1|.| 提|供|的产品|必须|符合|国家|及|行业标准|，|并通过|相关|质量|认证|。
  | |2|.| 具|备|高标准|的服务|体系|，|承诺|在|合作|期间|提供|持续|、|稳定|、|优质|的服务|。

四|、|行业|经验和|声誉|：
  | |1|.| 具|有|至少|五|年的|相关|行业|经验|。
  | |2|.| 有|良好的|业务|声誉|和|客户|满意度|记录|。

五|、|社会责任|和|可持续|性|：
  | |供应商|应|在其|经营|活动中|体现出|对社会|和|环境的|责任|，|以及|保证|业务的|可持续|性|。

六|、|其他|：
  | |供应商|还需|满足|其他|可能|的公司|特定|要求|，|以保证|合作的|顺利进行|。

只有|当|供应商|满足|上述|所有|标准|时|，|才有|资格|参与|投标|。||

'供应商要达到的标准主要包括以下六个方面：\n\n一、基本资质要求：\n   1. 必须具备合法有效的营业执照。\n   2. 必须具备税务登记证，且符合国家税收法律法规。\n   3. 应拥有有效的组织机构代码证。\n\n二、经营和财务能力：\n   1. 与公司合作的月份，销售的产品月总价不得低于人民币3万元。\n   2. 具备健全和稳定的财务状况，以及良好的信用记录。\n   3. 具备足够的流动资金来支持合同执行。\n\n三、产品和服务质量：\n   1. 提供的产品必须符合国家及行业标准，并通过相关质量认证。\n   2. 具备高标准的服务体系，承诺在合作期间提供持续、稳定、优质的服务。\n\n四、行业经验和声誉：\n   1. 具有至少五年的相关行业经验。\n   2. 有良好的业务声誉和客户满意度记录。\n\n五、社会责任和可持续性：\n   供应商应在其经营活动中体现出对社会和环境的责任，以及保证业务的可持续性。\n\n六、其他：\n   供应商还需满足其他可能的公司特定要求，以保证合作的顺利进行。\n\n只有当供应商满足上述所有标准时，才有资格参与投标。'

### ✍️ Excel 结构探查

In [764]:
import pandas as pd

In [765]:
def get_sheet_names(
        filename : str
) -> str :
    """获取 Excel 文件的工作表名称"""
    path = filename
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"
        
    excel_file = pd.ExcelFile(path.strip())
    sheet_names = excel_file.sheet_names
    return f"这是 '{path}' 文件的工作表名称：\n\n{sheet_names}"

In [766]:
def get_column_names(
        filename : str
) -> str:
    """获取 Excel 文件的列名"""

    # 读取 Excel 文件的第一个工作表
    path = filename
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"
        
    df = pd.read_excel(path.strip(), sheet_name=0)  # sheet_name=0 表示第一个工作表
    column_names = '\n'.join(
        df.columns.to_list()
    )

    result = f"这是 '{path.strip()}' 文件第一个工作表的列名：\n\n{column_names}"
    return result

In [772]:
def get_first_n_rows(
        filename : str,
        n : int = 3
) -> str :
    path = os.path.join(work_dir, filename)
    path = path.strip()
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"

    result = get_sheet_names(path)+"\n\n"
    result += get_column_names(path)+"\n\n"

    # 读取 Excel 文件的第一个工作表
    df = pd.read_excel(path, sheet_name=0)  # sheet_name=0 表示第一个工作表
    n_lines = '\n'.join(
        df.head(n).to_string(index=False, header=True).split('\n')
    )

    result += f"这是 '{path}' 文件第一个工作表的前{n}行样例：\n\n{n_lines}"
    return result

@tool
def inspect_excel(
        filename : str,
        n : int = 3
) -> str :
    """
    探查Excel数据文件的内容和结构，展示它的列名和前n行，n默认为3。
    注意，该工具仅使用于探查Excel文件，不能探查PDF或Word文件。
    
    使用该函数时应当准备提供filename和n两个参数，其中：
    
    - filename：要探查的Excel文件名
    - n: 默认的行数
    
    """
    return get_first_n_rows(filename, n)

In [773]:
##
print(inspect_excel({"filename": "2023年8月-9月销售记录.xlsx"}))

这是 './data/2023年8月-9月销售记录.xlsx' 文件的工作表名称：

['2023年8月-9月销售记录']

这是 './data/2023年8月-9月销售记录.xlsx' 文件第一个工作表的列名：

品类
产品名
单价(元)
销售量
销售日期
供应商

这是 './data/2023年8月-9月销售记录.xlsx' 文件第一个工作表的前3行样例：

   品类                产品名  单价(元)  销售量       销售日期        供应商
   手机       Xiaomi Mi 11   4999   20 2023-08-02   北京科技有限公司
   耳机    Sony WH-1000XM4   2999   15 2023-08-03   上海音响有限公司
笔记本电脑 Lenovo ThinkPad X1   8999   10 2023-08-05 深圳创新科技有限公司


### ✍️ Excel 数据分析

#### （1）准备

In [599]:
import re
from langchain.tools import StructuredTool
from langchain_core.output_parsers import BaseOutputParser

# from Utils.PythonExecUtil import execute_python_code
from langchain_openai import ChatOpenAI
from langchain_experimental.utilities import PythonREPL

#### （2）自定义一个OutputParse

In [735]:
class PythonCodeParser(BaseOutputParser):
    """从大模型返回的文本中提取Python代码。"""

    def _remove_marked_lines(self, input_str: str) -> str:
        lines = input_str.strip().split('\n')
        if lines and lines[0].strip().startswith('```'):
            del lines[0]
        if lines and lines[-1].strip().startswith('```'):
            del lines[-1]

        ans = '\n'.join(lines)
        return ans

    def parse(self, text: str) -> str:
        # 使用正则表达式找到所有的Python代码块
        python_code_blocks = re.findall(r'```python\n(.*?)\n```', text, re.DOTALL)
        # 从re返回结果提取出Python代码文本
        python_code = None
        if len(python_code_blocks) > 0:
            python_code = python_code_blocks[0]
            python_code = self._remove_marked_lines(python_code)
        return python_code

#### （3）定义提示语模板

In [749]:
from langchain.prompts import PromptTemplate

excel_analyser_prompt = PromptTemplate.from_template("""
你的任务是先分析，再生成代码。

请根据用户的输入，一步步分析：
（1）用户的输入是否依赖某个条件，而这个条件没有明确赋值？
（2）我是否需要对某个变量的值做假设？
（3）已经从用户的输入中拆解概念，将其中包含的数字或实体名称，映射为所生成的函数入参，并在代码中使用？
（4）不能生成用户输入中没有包含的函数入参，将导致严重后果，这一点是否已经确认？

如果我需要对某个变量的值做假设，请直接输出：
```python
print("我需要知道____的值，才能生成代码。请完善你的查询。") # 请将____替换为需要假设的的条件
```
否则，创建Python代码，分析指定文件的内容。

MUST 请不要使用filename作为入参变量，直接写死在代码里即可。

MUST 你生成代码中所有的常量都必须来自我给你的信息或来自文件本身。不要编造任何常量。
如果常量缺失，你的代码将无法运行。你可以拒绝生成代码，但是不要生成编造的代码。
确保你生成的代码最终以print的方式输出结果(回答用户的问题)。

MUST 你可以使用的库只包括：pandas, re, math, datetime, openpyxl
确保你的代码只使用上述库，否则你的代码将无法运行。

MUST 确保你的代码可以通过运行的。

给定文件为：
{filename}

文件内容样例：
{inspections}

你输出的Python代码前后必须有markdown标识符，如下所示：
```python
# example code
```

用户输入：
{query}
""")

#### （4）定义执行链

In [750]:
# llm = ChatOpenAI(
#         model="gpt-4-0125-preview",
#         temperature=0,
#         # model_kwargs={"seed": 42},
#     )
# analysis_chain = excel_analyser_prompt | llm | PythonCodeParser()

In [751]:
llm = ChatZhipuAI()
analysis_chain = excel_analyser_prompt | llm | PythonCodeParser()

#### （5）生成 python 代码并执行

In [752]:
import ast
import types
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_experimental.utilities import PythonREPL

@tool
def excel_analyse(query: str, filename: str):
    """如果给定一个Excel文件，就可以根据该工具分析其内容。"""

    path = os.path.join(work_dir, filename)
    path = path.strip()
    if(not os.path.exists(path)):
        return f"给定的文件路径不存在，请从工作目录{work_dir}中列举文件，确认其存在"

    # columns = get_column_names(filename)
    inspections = get_first_n_rows(filename)

    # 打印详细信息
    color_print("\n#!/usr/bin/env python", CODE_COLOR, end="\n")

    # 生成代码
    code = ""
    for c in analysis_chain.stream({
        "query": query,
        "filename": path,
        "inspections": inspections
    }):
        ## 打印详细信息
        color_print(c, CODE_COLOR, end="")
        ## 收集代码成果
        code += c

    if code:        
        # 执行代码
        return PythonREPL().run(code)
    else:
        return "没有找到可执行的Python代码"

In [753]:
##
excel_analyse.invoke({
    "query": "销售总额是多少?",
    "filename": "2023年8月-9月销售记录.xlsx"
})


#!/usr/bin/env python
# 导入库
import pandas as pd

# 读取Excel文件
df = pd.read_excel('./data/2023年8月-9月销售记录.xlsx', sheet_name='2023年8月-9月销售记录')

# 计算销售总额
total_sales = df['单价(元)'].multiply(df['销售量']).sum()

# 输出结果
print(f"销售总额是：{total_sales}元")

'销售总额是：5456735元\n'

### ✍️ 准备工具集

In [779]:
tools = [
    list_files,
    ask_document,
    inspect_excel,
    excel_analyse,
    FINISH,
]

for t in tools:
    print("-"*80)
    print("Tool: ", t.name)
    print("DESC: ", t.description)

--------------------------------------------------------------------------------
Tool:  list_files
DESC:  list_files(args=None) -> str - 如果需要查询资料，就首先使用该工具探查本地文件夹的结构和内容，展示它的文件名和文件夹名
--------------------------------------------------------------------------------
Tool:  ask_document
DESC:  ask_document(filename: str, query: str) -> str - 查询Word或PDF文档中的文本内容，以便回答问题。
    考虑上下文信息，确保问题对相关概念的定义表述完整。
--------------------------------------------------------------------------------
Tool:  inspect_excel
DESC:  inspect_excel(filename: str, n: int = 3) -> str - 探查Excel数据文件的内容和结构，展示它的列名和前n行，n默认为3。
    注意，该工具仅使用于探查Excel文件，不能探查PDF或Word文件。
    
    使用该函数时应当准备提供filename和n两个参数，其中：
    
    - filename：要探查的Excel文件名
    - n: 默认的行数
--------------------------------------------------------------------------------
Tool:  excel_analyse
DESC:  excel_analyse(query: str, filename: str) - 如果给定一个Excel文件，就可以根据该工具分析其内容。
--------------------------------------------------------------------------------
Tool:  FINISH
DESC: 

## 8、 🦜🔗LangChain 智能体对比

|langchain智能体|支持多工具|支持多参数|推理过程优化|
|:---|:---:|:---:|:---:|
|create_openai_tools_agent|支持|支持|依赖大模型能力|
|create_react_agent|不支持|不支持|可以支持|
|自定义CoT|可以支持|可以支持|可以支持|

### ✍️ GLM4 + create_openai_executor

In [812]:
glm_openai = create_openai_executor(
    ChatZhipuAI(model="glm-4"),
    [
        list_files,
        ask_document,
        inspect_excel,
        excel_analyse,
    ])

In [813]:
async for chunk in glm_openai.astream_events({"input": "供应商达标的业绩要求是什么？"}, version="v1"):
    if e['event'] in ["on_chat_model_end", "on_tool_end"]:
        if("input" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("INPUT:")
            print(e['data']['input'])
        if("output" in e['data']):
            print("\n", "-"*10, e['name'], "-"*2, e['event'], "-"*10)
            print("OUTPUT:")
            print(e['data']['output'])



> Entering new AgentExecutor chain...

Invoking: `list_files` with `{}`
responded: 这个问题涉及到具体的业绩要求和标准，我需要查询相关的文件或者资料。<|assistant|>

当前目录为： ./data
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
供应商资格要求.pdf
Invoking: `ask_document` with `{'filename': '供应商资格要求.pdf', 'query': '供应商达标的业绩要求是什么？'}`


供应商|达|标的|业绩|要求|主要包括|：

1|.| 月|销售额|：|与我|方|合作的|月份|，|经|我|方|销售|的产品|月|总价|不得|低于|人民币|3|万元|。
2|.| 稳|定的|财务|状况|：|供应商|应|具备|健全|和|稳定的|财务|状况|，|以及|良好的|信用|记录|。
3|.| 行|业|经验|：|供应商|应|具有|至少|五|年的|相关|行业|经验|。
4|.| 业务|声誉|：|供应商|应有|良好的|业务|声誉|和|客户|满意度|记录|。

这些|要求|旨在|确保|供应商|具备|足够的|经营|能力|、|财务|稳定性|、|行业|经验和|良好的|市场|声誉|，|以|保障|合作的|顺利进行|。||供应商达标的业绩要求主要包括：

1. 月销售额：与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元。
2. 稳定的财务状况：供应商应具备健全和稳定的财务状况，以及良好的信用记录。
3. 行业经验：供应商应具有至少五年的相关行业经验。
4. 业务声誉：供应商应有良好的业务声誉和客户满意度记录。

这些要求旨在确保供应商具备足够的经营能力、财务稳定性、行业经验和良好的市场声誉，以保障合作的顺利进行。根据我查到的信息，供应商达标的业绩要求主要包括月销售额、稳定的财务状况、行业经验和业务声誉等方面。具体来说，供应商需要满足与我方合作的月份，经我方销售的产品月总价不得低于人民币3万元的要求，同时应具备健全和稳定的财务状况，以及良好的信用记录。此外，供应商应具有至少五年的相关行业经验，并且有良好的业务声誉和客户满意度记录。这些要求有助于确保供应商具备足够的经营能力、财务稳定性、行业经验和良好的市场声誉，以保障合作的顺

### ✍️ GLM4 + create_react_agent

<div class="alert alert-warning">
    <b>思考：</b><br>
    为什么 langchain 内置 的 create_react_agent 无法解析多个输入参数？
</div>

**参考**

- [create_react_agent: output_parser](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/react/agent.py#L125)
- [ReActSingleInputOutputParser](https://github.com/langchain-ai/langchain/blob/239dd7c0c03d0430c55c2c41cf56cf0dd537199b/libs/langchain/langchain/agents/output_parsers/react_single_input.py)

In [672]:
zhipu_react = create_react_executor(
    ChatZhipuAI(),    [
        list_files,
        ask_document,
        get_first_n_rows,
        excel_analyse,
    ])

In [673]:
async for chunk in zhipu_react.astream_events({"input": "供应商达标的业绩要求是什么？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)



> Entering new AgentExecutor chain...
这个问题_涉及到_业绩_要求_，_我_需要_查看_相关的_文件_或_表格_以_获取_这些_信息_。_首先_，_我将_使用_ `_list__files_`_ 工_具_来_查看_是否有_相关的_文件_。

Thought_:_ 我_需要_查看_供应商_达标_业绩_要求_的相关_文件_。
Action_:_ list__files_
Action_ Input_:_ None_
Observ_ation__这个问题涉及到业绩要求，我需要查看相关的文件或表格以获取这些信息。首先，我将使用 `list_files` 工具来查看是否有相关的文件。

Thought: 我需要查看供应商达标业绩要求的相关文件。
Action: list_files
Action Input: None
Observation当前目录为： ./data
2023年8月-9月销售记录.xlsx
供应商名录.xlsx
供应商资格要求.pdf找到了_可能与_供应商_达标_业绩_要求_相关的_文件_，_我将_使用_ `_ask__doc_ment_`_ 工_具_来_查看_文件_ "_供应商_资格_要求_.pdf_"_。

Thought_:_ 我_将_使用_ `_ask__doc_ment_`_ 工_具_来_获取_ "_供应商_资格_要求_.pdf_"_ 文_件_中的_信息_。
Action_:_ ask__doc_ment_
Action_ Input_:_ filename_='_供应商_资格_要求_.pdf_',_ query_='_业绩_要求_'
Observ_ation__找到了可能与供应商达标业绩要求相关的文件，我将使用 `ask_docment` 工具来查看文件 "供应商资格要求.pdf"。

Thought: 我将使用 `ask_docment` 工具来获取 "供应商资格要求.pdf" 文件中的信息。
Action: ask_docment
Action Input: filename='供应商资格要求.pdf', query='业绩要求'
Observation

ValidationError: 1 validation error for ask_docmentSchema
query
  field required (type=value_error.missing)

### ✍️ GPT4 + create_cot_agent

In [821]:
gpt_cot = create_cot_executor(ChatOpenAI(model="gpt-4-0125-preview"), tools)

In [822]:

async for chunk in gpt_cot.astream_events({"input": "供应商达标的标准是什么？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

_```_json_
_{
_ _ "_name_":_ "_list__files_",
_ _ "_args_":_ {}
_}
_```__当前目录为： ./data
_```_json_
_{
_ _ "_name_":_ "_ask__document_",
_ _ "_args_":_ {
_   _ "_filename_":_ "_供_应_商_资_格_要_求_.pdf_",
_   _ "_query_":_ "_供_应_商_达_标_的_标_准_是_什_么_"
_ _ }
_}
_```__供应商|达标|的标准|如下|：

1|.| 基|本|资质|要求|：
  | -| 拥|有|合法|有效的|营业执照|。
  | -| 拥|有|税务|登记|证|，|并|符合|国家|税收|法律法规|。
  | -| 拥|有|有效的|组织|机构|代码|证|。

2|.| 经营|和|财务|能力|：
  | -| 与|公司|合作的|月份|，|销售|的产品|月|总价|不得|低于|人民币|3|万元|。
  | -| 具|备|健全|和|稳定的|财务|状况|，|以及|良好的|信用|记录|。
  | -| 具|备|足够的|流动|资金|来|支持|合同|执行|。

3|.| 产品|和服务|质量|：
  | -| 提|供|的产品|必须|符合|国家|及|行业标准|，|并通过|相关|质量|认证|。
  | -| 拥|有|高标准|的服务|体系|，|承诺|在|合作|期间|提供|持续|、|稳定|、|优质|的服务|。

4|.| 行|业|经验和|声誉|：
  | -| 具|有|至少|五|年的|相关|行业|经验|。
  | -| 拥|有|良好的|业务|声誉|和|客户|满意度|记录|。

5|.| 社|会|责任|和|可持续|性|：
  | -| 在|此|文档|中|未|具体|说明|，|但|通常|供应商|需要|展示|其对|社会责任|的|承诺|，|如|环保|、|节能|、|员工|福利|等方面的|实践|。

满足|以上|所有|条件的|供应商|，|方可|达到|投标|资质|要求|，|参与|招标|。||_```_json_
_{
_ _ "_name_":_ "_FIN_ISH_",
_ _ "_args_":_ {
_   _ "_output_":_ "_供_应_商_达_标_的_标_准_如_下_：_1_._ 基_本_资

In [783]:

async for chunk in gpt_cot.astream_events({"input": "9月份有哪些供货商达标？？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

_1_._ 关_键_概_念_:_ _供_货_商_达_标_（_供_货_商_在_9_月_份_是否_达_标_）
_2_._ 概_念_拆_解_:
_  _ -_ _供_货_商_达_标_
_    _ -_ 达_标_标_准_
_    _ -_ _供_货_商_在_9_月_份_的_供_货_数据_
_3_._ 反_思_:
_  _ -_ 需_要_找_到_记录_供_货_商_9_月_份_供_货_信息_的_文件_，_可能_是_Excel_格式_。
_  _ -_ 需_要_知_道_什_么_样_的_供_货_数据_算_是_达_标_。
_  _ -_ _之_前_没有_执行_过_任_何_动_作_，_所_以_还_没有_得_到_任_何_信息_。
_4_._ 思_考_:
_  _ -_ 需_要_先_确定_存_储_供_货_商_供_货_数据_的_文件_位置_和_名称_。
_  _ -_ 在_获取_文件_后_，_需要_查_看_文件_内容_，_确定_包_含_9_月_份_供_货_数据_的_列_。
_  _ -_ 需_要_定义_达_标_的_具_体_标_准_，_才_能_判断_供_货_商_是否_达_标_。
_  _ -_ 首_先_应_当_使用_list__files_工_具_来_查_找_有_关_供_货_商_供_货_信息_的_文件_。
_5_._ 推_理_:
_  _ -_ 应_首_先_使用_list__files_工_具_列_出_文件_，_以_便_找_到_可能_包_含_供_货_商_供_货_数据_的_文件_。
_  _ -_ _一_旦_找_到_相关_文件_，_可能_需要_使用_inspect__excel_工_具_来_查_看_文件_内容_。
_  _ -_ 如果_找_到_了_包_含_9_月_份_供_货_数据_的_文件_，_那_么_将_使用_excel__an_aly_se_工_具_来_分_析_数据_是否_达_标_。
_6_._ 计_划_:
_  _ -_ 使用_list__files_工_具_来_列_出_当前_目_录_下_所有_文件_，_以_寻_找_可能_包_含_9_月_份_供_货_商_供_货_数据_的_文件_。

_下_面_是_根_据_上_述_计_划_生成_的_JSON_格式_的_输出_：

_```_json_
_{"_name_":_ "_list__files_",_ "_args_":_ {}_}
_```_

### ✍️ GLM4 + create_cot_agent

In [780]:
zhipu_cot = create_cot_executor(ChatZhipuAI(model="glm-4"), tools)

In [781]:
async for chunk in zhipu_cot.astream_events({"input": "供应商达标的标准是什么？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

关键_概念_:
-_ _供应商_达标_标准_

概念_拆_解_:
-_ _供应商_达标_标准_ -_ _供应商_评价_体系_、_评价_标准_、_达标_分数线_

反思_:
-_ 在_解决_此_任务_之前_，_我_需要_了解_有关_供应商_评价_的具体_信息_，_但我_目前_并没有_这些_信息_。

思考_:
-_ 我_需要_先_获取_供应商_评价_体系_的相关_信息_，_才能_明确_达标_标准_。
-_ _供应商_评价_体系和_达标_标准_可能_存在于_Excel_或_Word_文档_中_。
-_ 我_应当_首先_查找_可能_含有_这些_信息的_文件_，_然后_分析_文件_内容_以_获取_所需_信息_。

推理_:
-_ 根_据_以上_思考_，_我_需要_先_使用_`_list__files_`_工具_来_查询_本地_文件夹_，_找到_可能_含有_供应商_评价_体系和_达标_标准的_文件_。

计划_:
-_ 我_将_使用_`_list__files_`_工具_来_查询_本地_文件夹_，_寻找_与_供应商_评价_相关的_文件_。

输出_动作_:

```_json_
{"_name_":_ "_list__files_",_ "_args_":_ {}_}
```__当前目录为： ./data
关键_概念_:_ _供应商_达标_标准_

概念_拆_解_:
-_ _供应商_达标_标准_ -_ _供应商_评价_体系_、_评价_标准_、_达标_分数线_

反思_:
-_ 我_目前_还没有_获取_到_供应商_评价_体系_的相关_信息_，_这是_解决_此_任务_的关键_。

思考_:
-_ 我_需要_先_找到_包含_供应商_评价_体系和_达标_标准的_文件_。
-_ 已经_查询_到_本地_文件夹_中有_"_供应商_资格_要求_.pdf_"，_这可能_包含_达标_标准_的信息_。
-_ 我_计划_使用_`_ask__document_`_工具_来_查询_这个_PDF_文件_。

推理_:
-_ 根_据_以上_思考_，_我将_使用_`_ask__document_`_工具_来_查询_"_供应商_资格_要求_.pdf_"，_以_获取_供应商_达标_标准_。

计划_:
-_ 我_将_使用_`_ask__document_`_工具_，_查询_"_供应商_资格_要求_.pdf_"_文

In [782]:
async for chunk in zhipu_cot.astream_events({"input": "9月份有哪些供货商达标？"}, version="v1"):
    event = chunk['event']
    if(event == "on_chat_model_stream"):
        if('chunk' in chunk['data']):
            print(chunk['data']['chunk'].content, end="_", flush=True)

关键_概念_:
-_ _9_月份_供货_商_达标_情况_

概念_拆_解_:
-_ _供货_商_达标_情况_
 _ -_ -_ _供货_商_名称_
 _ -_ -_ 达_标_月份_
 _ -_ -_ 达_标_标准_

反思_:
-_ 需_要_查询_9_月份_的_供货_商_达标_情况_，_首先_需要_知道_达标_的标准_是什么_，_然后再_查看_哪些_供货_商_达到了_这个_标准_。
-_ 目前_我_还没有_获得_任何_关于_供货_商_达标_情况_的信息_。

思考_:
-_ 为了_获取_9_月份_供货_商_达标_情况_，_我_需要_先_了解_达标_标准_，_然后_才能_确定_哪些_供货_商_达到了_这个_标准_。
-_ 我_可以通过_查询_Excel_文件_来_获取_这些_信息_，_因为_Excel_文件_通常_用于_存储_这类_数据_。

计划_:
-_ 我_将_首先_使用_"_inspect__excel_"_工具_探_查_存储_供货_商_达标_信息的_Excel_文件_，_以便_了解_其_结构和_内容_。

输出_JSON_格式_描述_的动作_:

```_json_
{"_name_":_ "_inspect__excel_",_ "_args_":_ {"_filename_":_ "_supplier__stand_ards_.xlsx_",_ "_n_":_ _3_}}
```__关键_概念_:_ _9_月份_供货_商_达标_情况_
概念_拆_解_:
-_ _供货_商_达标_情况_
 _ -_ -_ _供货_商_名称_
 _ -_ -_ 达_标_月份_
 _ -_ -_ 达_标_标准_

反思_:
-_ 需_要_获取_9_月份_供货_商_的_达标_情况_，_但_尚未_了解_达标_标准_。
-_ 上_一个_计划_中_使用的_文件_路径_不存在_，_需要_先_探_查_正确的_文件_路径_。

思考_:
-_ 需_要先_确定_正确的_文件_路径_和_文件_名_，_然后再_探_查_Excel_文件_了解_达标_标准和_供货_商_的_达标_情况_。
-_ 可以_使用_list__files_工具_来_查找_存储_供货_商_达标_信息的_文件_。

计划_:
-_ 使用_list__files_工具_探_查_工作_目录_./_data_中的_文件_，_找到_存储_

Stopping agent prematurely due to triggering stop condition


当前目录为： ./data


## ❤️ 知识点小结

- 解构源码 create_openai_tools_agent 并实践
- 解构源码 create_react_agent 并实践
- 学习如何自定义符合 AgentExecutor 的 AutoGPT 智能体

**以及：**

|langchain智能体|支持多工具|支持多参数|推理过程优化|
|:---|:---:|:---:|:---:|
|create_openai_tools_agent|支持|支持|依赖大模型能力|
|create_react_agent|不支持|不支持|可以支持|
|自定义CoT|可以支持|可以支持|可以支持|

# 结束了

## ❤️ 彩蛋

**langgraph 比 LCEL 增加的能力**
- 增加对智能体逻辑控制的能力
- 增加全局状态管理（智能体和工具都可以参与管理）
- 增加对记忆的深度管理
- 增加对工具的流式输出控制

**Langgraph中一些较新的智能体论文实践...**

- [Agent Supervisor](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/agent_supervisor.ipynb)
- [Hierarchical Agent Teams](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/hierarchical_agent_teams.ipynb)
- [Multi Agent Collaboration](https://github.com/langchain-ai/langgraph/blob/main/examples/multi_agent/multi-agent-collaboration.ipynb)
- [Plan And Execute](https://github.com/langchain-ai/langgraph/blob/main/examples/plan-and-execute/plan-and-execute.ipynb)
- [ReWoo](https://github.com/langchain-ai/langgraph/blob/main/examples/rewoo/rewoo.ipynb)
- [Reflexion](https://github.com/langchain-ai/langgraph/blob/main/examples/reflexion/reflexion.ipynb)
- [Self Discover](https://github.com/langchain-ai/langgraph/blob/main/examples/self-discover/self-discover.ipynb)
- ...